# Stabilized FEM for the Stokes problem

On the unit square domain $\Omega$ we consider the Stokes problem expressed by 
\begin{align*} 
- \boldsymbol{\Delta}\boldsymbol{u} +\boldsymbol{\nabla}  p  &= \boldsymbol{f} \quad {\rm in} \ \Omega, \\ 
\boldsymbol{\nabla} \cdot \boldsymbol{u} &= 0 \quad {\rm in} \ \Omega, \\ 
\boldsymbol{u} &= \boldsymbol{u}_D \;\; {\rm on} \ \Gamma_D, \\
(\boldsymbol{\nabla} \boldsymbol{u} - p)\cdot\boldsymbol{n} &= \boldsymbol{g}_N\;\; {\rm on}\ \partial\Omega \setminus \Gamma_D.
\end{align*}

## Saddle-point weak formulation

The weak formulation is derived by testing the first equation with a vector-valued function $\boldsymbol{v}$ in $H^1_0(\Omega)^d$ and the second equation with $q\in L^2(\Omega)$. Assuming, for the sake of simplicity, that $\boldsymbol{u}_D = \boldsymbol{0}$, and integrating by parts in the first equation, we end up with the saddle point problem
\begin{align*}
\int_{\Omega} \boldsymbol{\nabla} \boldsymbol{u} \cdot \boldsymbol{\nabla} \boldsymbol{v}
- (\boldsymbol{\nabla} \cdot \boldsymbol{v}) \ p \ {\rm d} x
&= \int_{\Omega} \boldsymbol{f} \cdot \boldsymbol{v} \, {\rm d} x + \int_{\Gamma_N} \boldsymbol{g} \cdot \boldsymbol{v} \, {\rm d} s. \\
\int_\Omega (\boldsymbol{\nabla} \cdot \boldsymbol{u}) \ q \ {\rm d} x &= 0
\end{align*}

Defining $a: H^1_0(\Omega)^d\times H^1_0(\Omega)^d$ and $b:H^1_0(\Omega)^d\times L^2(\Omega)$ such that
\begin{align*}
a(\boldsymbol{v},\boldsymbol{w}) &:=\int_{\Omega} \boldsymbol{\nabla} \boldsymbol{v} \cdot \boldsymbol{\nabla} \boldsymbol{w}\  {\rm d} x, 
\quad\forall\ v,w\in H^1_0(\Omega)^d \\ 
b(\boldsymbol{v},q) &:= -\int_\Omega (\boldsymbol{\nabla} \cdot \boldsymbol{v}) \ q \ {\rm d} x, \quad\forall\ v\in H^1_0(\Omega)^d \;\text{and}\; \forall\ q\in L^2(\Omega),
\end{align*}
the weak form of Stokes problem reads: Find $(u,p)$ such that
\begin{align*}
a(\boldsymbol{u},\boldsymbol{v}) + b(\boldsymbol{v},p)
&= \int_{\Omega} \boldsymbol{f} \cdot \boldsymbol{v} \, {\rm d} x + \int_{\Gamma_N} \boldsymbol{g} \cdot \boldsymbol{v} \, {\rm d} s. \\
b(\boldsymbol{u},q) &= 0.
\end{align*}

## Numerical solution with FEniCS

In [ ]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
from fenics import *
from mshr import *
import matplotlib.pyplot as plt
import numpy as np

We want to compare the **stabilized $\mathbb{P}^1$-$\mathbb{P}^1$ FEM** and the **equal-order discontinuous Galerkin (dG) method** for Stokes with respect to two other first order stable elements, i.e. the **MINI element** and the **Crouzeix--Raviart element**.

#### FE solution with MINI element

In [ ]:
def mini_stokes(n, u_exact, p_exact, f, gNt, gNr):    
    # 1. generate the mesh and mark the boundaries
    mesh = UnitSquareMesh(n, n, 'crossed')
    
    boundary_markers = MeshFunction('size_t', mesh, mesh.geometric_dimension()-1, 0)

    class top_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[1], 1)

    class right_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], 1)

    class other_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return near(x[0], 0) or near(x[1], 0)

    top = top_boundary()
    top.mark(boundary_markers, 1)
    right = right_boundary()
    right.mark(boundary_markers, 2)
    other = other_boundary()
    other.mark(boundary_markers, 3)
    
    ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

    # 2. finite element spaces and Dirichlet BC
    Q = FiniteElement('CG', mesh.ufl_cell(), 1)
    B = FiniteElement("Bubble", mesh.ufl_cell(), mesh.topology().dim() + 1)
    V = VectorElement(NodalEnrichedElement(Q, B))
    X = FunctionSpace(mesh, V * Q)

    bc = DirichletBC(X.sub(0), u_exact, boundary_markers, 3)

    # 3. problem definition
    u, p = TrialFunctions(X)
    v, q = TestFunctions(X)

    a = (inner(grad(u), grad(v)) - p*div(v) - div(u)*q) * dx
    L = dot(f, v) * dx + dot(gNt, v) * ds(1) + dot(gNr, v) * ds(2)

    # 4. solution
    x = Function(X)
    solve(a == L, x, bc)

    u, p = x.split()
    return u, p

#### FE solution with Crouzeix-Raviart element

In [ ]:
def cr_stokes(n, u_exact, p_exact, f, gNt, gNr):    
    # 1. generate the mesh and mark the boundaries
    mesh = UnitSquareMesh(n, n, 'crossed')
    
    boundary_markers = MeshFunction('size_t', mesh, mesh.geometric_dimension()-1, 0)

    class top_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[1], 1)

    class right_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], 1)

    class other_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return near(x[0], 0) or near(x[1], 0)

    top = top_boundary()
    top.mark(boundary_markers, 1)
    right = right_boundary()
    right.mark(boundary_markers, 2)
    other = other_boundary()
    other.mark(boundary_markers, 3)
    
    ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

    # 2. finite element space and Dirichlet BC
    V = VectorElement('CR', mesh.ufl_cell(), 1)
    Q = FiniteElement('DG', mesh.ufl_cell(), 0)
    X = FunctionSpace(mesh, V * Q)

    bc = DirichletBC(X.sub(0), u_exact, boundary_markers, 3)

    # 3. problem definition
    u, p = TrialFunctions(X)
    v, q = TestFunctions(X)

    a = (inner(grad(u), grad(v)) - p*div(v) - div(u)*q) * dx
    L = dot(f, v) * dx + dot(gNt, v) * ds(1) + dot(gNr, v) * ds(2)

    # 4. solution
    x = Function(X)
    solve(a == L, x, bc)

    u, p = x.split()
    return u, p

#### FE solution with discontinuous Galerkin element

In [ ]:
def dg_stokes(n, deg, u_exact, p_exact, f, gNt, gNr):    
    # 1. generate the mesh and mark the boundaries
    mesh = UnitSquareMesh(n, n, 'crossed')
    
    boundary_markers = MeshFunction('size_t', mesh, mesh.geometric_dimension()-1, 0)

    class top_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[1], 1)

    class right_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], 1)

    class other_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return near(x[0], 0) or near(x[1], 0)

    top = top_boundary()
    top.mark(boundary_markers, 1)
    right = right_boundary()
    right.mark(boundary_markers, 2)
    other = other_boundary()
    other.mark(boundary_markers, 3)
    
    ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

    # 2. finite element space and Dirichlet BC
    V = VectorElement('DG', mesh.ufl_cell(), deg)
    Q = FiniteElement('DG', mesh.ufl_cell(), deg)
    X = FunctionSpace(mesh, V * Q)

    # 3. problem definition
    u, p = TrialFunctions(X)
    v, q = TestFunctions(X)
    n = FacetNormal(mesh)
    h = CellDiameter(mesh)
    h_avg = (h('+') + h('-'))/2
    alpha = 10.1*deg**2
    beta = 3.1/deg

    # Define forms
    a = (inner(grad(u), grad(v)) - p*div(v) + div(u)*q)*dx  \
      - inner(jump(u, n), avg(q))*dS + inner(jump(v, n), avg(p))*dS \
      - inner(u, n)*q*ds(3) + inner(v, n)*p*ds(3) \
      - inner(dot(avg(grad(u)), n('+')), jump(v))*dS \
      - inner(jump(u), dot(avg(grad(v)), n('+')))*dS \
      - inner(dot(grad(u), n), v)*ds(3) \
      - inner(u, dot(grad(v), n))*ds(3) \
      + alpha/h_avg*inner(jump(u), jump(v))*dS \
      + alpha/h*inner(u, v)*ds(3) \
      + beta*h_avg*jump(p)*jump(q)*dS 

    L = dot(f, v) * dx + dot(gNt, v) * ds(1) + dot(gNr, v) * ds(2)

    # 4. solution
    x = Function(X)
    solve(a == L, x)

    u, p = x.split()
    return u, p

#### FE solution with SUPG stabilized P1-P1 element

In [ ]:
def supg_stokes(n, u_exact, p_exact, f, gNt, gNr):    
    # 1. generate the mesh and mark the boundaries
    mesh = UnitSquareMesh(n, n, 'crossed')
    
    boundary_markers = MeshFunction('size_t', mesh, mesh.geometric_dimension()-1, 0)

    class top_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[1], 1)

    class right_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], 1)

    class other_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return near(x[0], 0) or near(x[1], 0)

    top = top_boundary()
    top.mark(boundary_markers, 1)
    right = right_boundary()
    right.mark(boundary_markers, 2)
    other = other_boundary()
    other.mark(boundary_markers, 3)
    
    ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

    # 2. finite element space and Dirichlet BC
    V = VectorElement('CG', mesh.ufl_cell(), 1)
    Q = FiniteElement('CG', mesh.ufl_cell(), 1)
    X = FunctionSpace(mesh, V * Q)

    bc = DirichletBC(X.sub(0), u_exact, boundary_markers, 3)

    # 3. problem definition
    u, p = TrialFunctions(X)
    v, q = TestFunctions(X)

    a = (inner(grad(u), grad(v)) - p*div(v) + div(u)*q) * dx
    L = dot(f, v) * dx + dot(gNt, v) * ds(1) + dot(gNr, v) * ds(2)
  
    h = CellDiameter(X.mesh())
    tau_K = 0.5 * (h**2) 
    a += tau_K * (inner(grad(p), grad(q)) + div(u)*div(v)) * dx
    L += tau_K * dot(f,grad(q)) * dx  

    # 4. solution
    x = Function(X)
    solve(a == L, x, bc)

    u, p = x.split()
    return u, p

#### FE solution with P1-P1 element stabilized with pressure mass matrix

In [ ]:
def massstab_stokes(n, u_exact, p_exact, f, gNt, gNr):    
    # 1. generate the mesh and mark the boundaries
    mesh = UnitSquareMesh(n, n, 'crossed')
    
    boundary_markers = MeshFunction('size_t', mesh, mesh.geometric_dimension()-1, 0)

    class top_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[1], 1)

    class right_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], 1)

    class other_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return near(x[0], 0) or near(x[1], 0)

    top = top_boundary()
    top.mark(boundary_markers, 1)
    right = right_boundary()
    right.mark(boundary_markers, 2)
    other = other_boundary()
    other.mark(boundary_markers, 3)
    
    ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

    # 2. finite element space and Dirichlet BC
    V = VectorElement('CG', mesh.ufl_cell(), 1)
    Q = FiniteElement('DG', mesh.ufl_cell(), 0)
    X = FunctionSpace(mesh, V * Q)

    bc = DirichletBC(X.sub(0), u_exact, boundary_markers, 3)

    # 3. problem definition
    u, p = TrialFunctions(X)
    v, q = TestFunctions(X)

    a = (inner(grad(u), grad(v)) - p*div(v) + div(u)*q) * dx
    L = dot(f, v) * dx + dot(gNt, v) * ds(1) + dot(gNr, v) * ds(2)

    h = CellDiameter(mesh)
    a_stab = a + (h**2) * p * q * dx

    # 4. solution
    x = Function(X)
    solve(a_stab == L, x, bc)

    u, p = x.split()
    return u, p

### Convergence analysis and comparizon of the different methods

Compute the functions $\boldsymbol{f}$, $\boldsymbol{g}_N$ and $\boldsymbol{u}_D$ corresponding to the analytical solution
$$
            \boldsymbol{u}(x,y) = \begin{bmatrix}(1-\cos(\pi x))\sin(\pi y) \\ \sin(\pi x)(\cos(\pi y)-1)\end{bmatrix},
            \qquad
            p(x,y) = -\frac{1}{4}(\cos{(2\pi x)}+\cos{(2\pi y)}).
$$

Assess the convergence, measuring the errors as 
$$
||\boldsymbol{u} - \boldsymbol{u}_h||_{H^1(\Omega)}, \quad\text{ and}\quad ||p - p_h||_ {L^2(\Omega)}.
$$

In [ ]:
u_exact = Expression((
        '(1.0-cos(pi*x[0])) * sin(pi*x[1])',
        'sin(pi*x[0]) * (cos(pi*x[1])-1.0)'
                    ), degree=3)
p_exact = Expression(
        '-0.25 * (cos(2*pi*x[0]) + cos(2*pi*x[1]))',
        degree=3)
f = Expression((
        '-2*pi*pi*cos(pi*x[0])*sin(pi*x[1]) + 0.5*pi*sin(2*pi*x[0]) + pi*pi*sin(pi*x[1])',
        '2*pi*pi*sin(pi*x[0])*cos(pi*x[1]) + 0.5*pi*sin(2*pi*x[1]) - pi*pi*sin(pi*x[0])'
               ), degree=2)

gNt = Expression(('pi * (cos(pi*x[0])-1.0)', '0.25 * (cos(2*pi*x[0]) + cos(2*pi*x[1]))'
                 ), degree=2)
gNr = Expression(('0.25 * (cos(2*pi*x[0]) + cos(2*pi*x[1]))', 'pi * (1.0-cos(pi*x[1]))'
                 ), degree=2)

for n in [10, 20, 40, 80]:
    uh, ph = mini_stokes(n, u_exact, p_exact, f, gNt, gNr)
    
    eL2 = errornorm(p_exact, ph, 'L2')
    eH1 = errornorm(u_exact, uh, 'H1')

    print('n={} eL2={:.2e} eH1={:.2e}'.format(n, eL2, eH1))

print()
for n in [10, 20, 40, 80]:
    uh, ph = cr_stokes(n, u_exact, p_exact, f, gNt, gNr)
    
    eL2 = errornorm(p_exact, ph, 'L2')
    eH1 = errornorm(u_exact, uh, 'H1')

    print('n={} eL2={:.2e} eH1={:.2e}'.format(n, eL2, eH1))

print()
for n in [10, 20, 40, 80]:
    uh, ph = supg_stokes(n, u_exact, p_exact, f, gNt, gNr)
    
    eL2 = errornorm(p_exact, ph, 'L2')
    eH1 = errornorm(u_exact, uh, 'H1')

    print('n={} eL2={:.2e} eH1={:.2e}'.format(n, eL2, eH1))

print()
degree = 1
for n in [10, 20, 40, 80]:
    uh, ph = dg_stokes(n, degree, u_exact, p_exact, f, gNt, gNr)
    
    eL2 = errornorm(p_exact, ph, 'L2')
    eH1 = errornorm(u_exact, uh, 'H1')

    print('n={} eL2={:.2e} eH1={:.2e}'.format(n, eL2, eH1))

n=10 eL2=8.70e-02 eH1=3.27e-01
n=20 eL2=3.09e-02 eH1=1.62e-01
n=40 eL2=1.09e-02 eH1=8.08e-02
n=80 eL2=3.86e-03 eH1=4.03e-02

n=10 eL2=1.18e-01 eH1=3.45e-01
n=20 eL2=5.92e-02 eH1=1.74e-01
n=40 eL2=2.97e-02 eH1=8.72e-02
n=80 eL2=1.49e-02 eH1=4.37e-02

n=10 eL2=3.98e-01 eH1=5.18e-01
n=20 eL2=1.47e-01 eH1=2.24e-01
n=40 eL2=5.28e-02 eH1=1.01e-01
n=80 eL2=1.89e-02 eH1=4.72e-02

n=10 eL2=9.56e-02 eH1=3.20e-01
n=20 eL2=3.77e-02 eH1=1.58e-01
n=40 eL2=1.27e-02 eH1=7.79e-02
n=80 eL2=4.43e-03 eH1=3.88e-02
